In [ ]:
!pip install datasets
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from datasets import load_dataset
import numpy as np
import tiktoken
import torch
import sys

data = load_dataset("imdb")
train_data=data['train'].data.to_pydict()
val_data=data['test'].data.to_pydict()
tokenizer=tiktoken.encoding_for_model("gpt-4")
class Dataset(torch.utils.data.Dataset):
  def __init__(self,data,tokenizer):
    self.data = data
    self.tokenizer=tokenizer

  def __getitem__(self,sample_n):
    tokens=self._pad_or_trim(
        self.tokenizer.encode(self.data['text'][sample_n])
    )
    label=self.data['label'][sample_n]
    return tokens, label

  def __len__(self):
    return len(self.data['label'])

  def _pad_or_trim(self,tokens):
    output=np.array([self.tokenizer.eot_token]*512)
    tokens=tokens[:512]
    output[:len(tokens)]=tokens
    return output

train_dataset=Dataset(train_data,tokenizer)
val_dataset=Dataset(val_data,tokenizer)

train_dataloader=torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=1

)
val_dataloader=torch.utils.data.DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=1

)

class Model(torch.nn.Module):
  def __init__(self,max_token_value):
    super().__init__()
    self.embedding=torch.nn.Embedding(
        max_token_value,1024,padding_idx=tokenizer.eot_token
    )
    self.classifier=torch.nn.ModuleList([
        torch.nn.Linear(1024,1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024,1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024,2)
    ])
  def forward(self,x):
    x=self.embedding(x)
    x=x.mean(1)
    for layer in self.classifier:
      x=layer(x)
    return x

model=Model(tokenizer.eot_token+1).cuda()


loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=1e-3)
best_val_acc=0
for epoch_n in range(5):
  model.train()
  training_loss=[]
  for tokens,labels in train_dataloader:
    output=model(tokens.cuda())
    loss=loss_fn(output,labels.cuda())
    training_loss.append(loss.detach().cpu().item())
    loss.backward()
    optimizer.step()
    model.zero_grad()
  model.eval()
  all_predictions=[]
  all_labels=[]
  for tokens,labels in val_dataloader:
    with torch.no_grad():
      output=model(tokens.cuda())
    predictions=output.argmax(-1)
    all_predictions +=predictions.detach().cpu().tolist()
    all_labels += labels.tolist()
  val_accuracy=np.mean(np.array(all_labels)==np.array(all_predictions))

  print(f'Epoch {epoch_n}, training_loss: {np.mean(training_loss)}, val_accuracy: {val_accuracy}')

  torch.save(model, './latest_model.ckpt')
  torch.save(optimizer, './latest_optimizer.ckpt')

  if val_accuracy > best_val_acc:
    torch.save(model, './best_model.ckpt')

#model loading
model=torch.load('./best_model.ckpt')
model.eval()
all_predictions=[]
all_labels=[]
for tokens,labels in val_dataloader:
  with torch.no_grad():
    output=model(tokens.cuda())
  predictions=output.argmax(-1)
  all_predictions+=predictions.detach().cpu().tolist()
  all_labels+=labels.tolist()
val_accuracy=np.mean(np.array(all_labels)==np.array(all_predictions))
print(f'Loaded model accuracy: {val_accuracy}')

Epoch 0, training_loss: 0.5591632420386468, val_accuracy: 0.79196
